In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import keras
import tensorflow as tf
from keras import layers

In [22]:
data = pd.read_csv("Churn Modeling.csv")

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [24]:
data.corr()

<ipython-input-24-c44ded798807>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248
CreditScore,0.005840,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094
Age,0.000783,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323
Tenure,-0.006495,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097


In [25]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
RowNumber,10000.0,5.000500e+03,2886.895680,1.00,2500.75,5.000500e+03,7.500250e+03,10000.00
CustomerId,10000.0,1.569094e+07,71936.186123,15565701.00,15628528.25,1.569074e+07,1.575323e+07,15815690.00
CreditScore,10000.0,6.505288e+02,96.653299,350.00,584.00,6.520000e+02,7.180000e+02,850.00
Age,10000.0,3.892180e+01,10.487806,18.00,32.00,3.700000e+01,4.400000e+01,92.00
Tenure,10000.0,5.012800e+00,2.892174,0.00,3.00,5.000000e+00,7.000000e+00,10.00
Balance,10000.0,7.648589e+04,62397.405202,0.00,0.00,9.719854e+04,1.276442e+05,250898.09
NumOfProducts,10000.0,1.530200e+00,0.581654,1.00,1.00,1.000000e+00,2.000000e+00,4.00
HasCrCard,10000.0,7.055000e-01,0.455840,0.00,0.00,1.000000e+00,1.000000e+00,1.00
IsActiveMember,10000.0,5.151000e-01,0.499797,0.00,0.00,1.000000e+00,1.000000e+00,1.00
EstimatedSalary,10000.0,1.000902e+05,57510.492818,11.58,51002.11,1.001939e+05,1.493882e+05,199992.48


In [26]:
data.loc[:, "Liquidity"] = (data.loc[:, "EstimatedSalary"]) / (data.loc[:, "Balance"])

In [27]:
for i in range(0, len(data.loc[:, "Liquidity"])):
  #print(data.loc[i, "Liquidity"])
  if data.loc[i, "Liquidity"] == float('inf'):
    data.loc[i, "Liquidity"] = 0

'for i in range(0, len(data.loc[:, "Liquidity"])):\n  #print(data.loc[i, "Liquidity"])\n  if data.loc[i, "Liquidity"] == float(\'inf\'):\n    data.loc[i, "Liquidity"] = 0\n'

In [28]:
# Generations

#2023 - 92

"""
Silent Generation  = 1928 - 1945 0
Baby Boomers = 1946 - 1964 1
Generation X = 1965 - 1980 2
Millenials = 1981 - 1996 3
Generation Z = 1997 - 2012 4
Generation Alpha = 2013 - present
"""

current_year = 2023

silent_upper_bound = current_year - 1928
silent_lower_bound = current_year - 1945
boomer_upper_bound = current_year - 1946
boomer_lower_bound = current_year - 1964
genx_upper_bound = current_year - 1965
genx_lower_bound = current_year - 1980
millenial_upper_bound = current_year - 1981
millenial_lower_bound = current_year - 1996
genz_upper_bound = current_year - 1997
genz_lower_bound = current_year - 2012

In [29]:
print(millenial_upper_bound , millenial_lower_bound)

42 27


In [30]:
data.loc[:, "Generation"] = (data.loc[:, "Age"]) >= 78

In [31]:
def generations(row):
  generation_index = 0
  if row >= 78:
    generation_index = 0
  elif row >= 59 and row < 78:
    generation_index = 1
  elif row >= 43 and row < 59:
    generation_index = 2
  elif row >= 27 and row <= 42:
    generation_index = 3
  elif row < 27:
    generation_index = 4
  return generation_index

In [32]:
data.loc[:, "Generation"] = data.loc[:, "Age"].apply(generations)

In [33]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,3
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,3
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,3
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,3
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,2


In [34]:
silent_generation_not_churned = (data.loc[:, "Exited"] == 0) & (data.loc[:, "Generation"] == 0)

In [35]:
len(data.loc[:, "Generation"][silent_generation_not_churned])

23

In [36]:
silent_generation_churned = (data.loc[:, "Exited"] == 1) & (data.loc[:, "Generation"] == 0)

In [37]:
len(data.loc[:, "Generation"][silent_generation_churned])

1

In [38]:
len(data[data.loc[:, "Age"] >= 78])

24

In [83]:
boomer_generation_not_churned = (data.loc[:, "Exited"] == 0) & (data.loc[:, "Generation"] == 1)
len(data.loc[:, "Generation"][boomer_generation_not_churned])

388

In [84]:
boomer_generation_churned = (data.loc[:, "Exited"] == 1) & (data.loc[:, "Generation"] == 1)
len(data.loc[:, "Generation"][boomer_generation_churned])

176

In [85]:
genx_not_churned = (data.loc[:, "Exited"] == 0) & (data.loc[:, "Generation"] == 2)
len(data.loc[:, "Generation"][genx_not_churned])

1278

In [86]:
genx_churned = (data.loc[:, "Exited"] == 1) & (data.loc[:, "Generation"] == 2)
len(data.loc[:, "Generation"][genx_churned])

1028

In [87]:
millenials_not_churned = (data.loc[:, "Exited"] == 0) & (data.loc[:, "Generation"] == 3)
len(data.loc[:, "Generation"][millenials_not_churned])

5523

In [88]:
millenials_churned = (data.loc[:, "Exited"] == 1) & (data.loc[:, "Generation"] == 3)
len(data.loc[:, "Generation"][millenials_churned])

772

In [89]:
genz_not_churned = (data.loc[:, "Exited"] == 0) & (data.loc[:, "Generation"] == 4)
len(data.loc[:, "Generation"][genz_not_churned])

751

In [90]:
genz_churned = (data.loc[:, "Exited"] == 1) & (data.loc[:, "Generation"] == 4)
len(data.loc[:, "Generation"][genz_churned])

60

In [41]:
print(data.loc[:, "Liquidity"])

In [42]:
data.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0,3
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0,3
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1,3
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1,3
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0,3


In [43]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,3
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,3
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,3
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,3
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,2


In [44]:
data[0:200]

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,3
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,3
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,3
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,3
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,15594815,Aleshire,807,France,Male,35,3,174790.15,1,1,1,600.36,0,3
196,197,15635905,Moran,616,Spain,Female,32,6,0.00,2,1,1,43001.46,0,3
197,198,15777892,Samsonova,721,Germany,Male,37,3,107720.64,1,1,1,158591.12,0,3
198,199,15656176,Jenkins,501,France,Male,57,10,0.00,2,1,1,47847.19,0,2


In [45]:
print(data.columns)

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Generation'],
      dtype='object')


In [46]:
data = data[['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Liquidity', 'Generation', 'Exited']]

"data = data[['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',\n       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',\n       'IsActiveMember', 'EstimatedSalary', 'Liquidity', 'Exited']]\n"

In [47]:
data.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Generation'],
      dtype='object')

In [48]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700,2.730500
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769,0.697365
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000,2.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000,3.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000,3.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000,4.000000


In [49]:
data.corr()

<ipython-input-49-c44ded798807>:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr()


,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
RowNumber,1.000000,0.004202,0.005840,0.000783,-0.006495,-0.009067,0.007246,0.000599,0.012044,-0.005988,-0.016571,-0.004566
CustomerId,0.004202,1.000000,0.005308,0.009497,-0.014883,-0.012419,0.016972,-0.014025,0.001665,0.015271,-0.006248,-0.009395
CreditScore,0.005840,0.005308,1.000000,-0.003965,0.000842,0.006268,0.012238,-0.005458,0.025651,-0.001384,-0.027094,0.004308
Age,0.000783,0.009497,-0.003965,1.000000,-0.009997,0.028308,-0.030680,-0.011721,0.085472,-0.007201,0.285323,-0.912072
Tenure,-0.006495,-0.014883,0.000842,-0.009997,1.000000,-0.012254,0.013444,0.022583,-0.028362,0.007784,-0.014001,0.011975
Balance,-0.009067,-0.012419,0.006268,0.028308,-0.012254,1.000000,-0.304180,-0.014858,-0.010084,0.012797,0.118533,-0.023566
NumOfProducts,0.007246,0.016972,0.012238,-0.030680,0.013444,-0.304180,1.000000,0.003183,0.009612,0.014204,-0.047820,0.028820
HasCrCard,0.000599,-0.014025,-0.005458,-0.011721,0.022583,-0.014858,0.003183,1.000000,-0.011866,-0.009933,-0.007138,0.007652
IsActiveMember,0.012044,0.001665,0.025651,0.085472,-0.028362,-0.010084,0.009612,-0.011866,1.000000,-0.011421,-0.156128,-0.073114
EstimatedSalary,-0.005988,0.015271,-0.001384,-0.007201,0.007784,0.012797,0.014204,-0.009933,-0.011421,1.000000,0.012097,0.006819


In [50]:
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [51]:
mask_credit_score = (data.loc[:, "CreditScore"] <= 584)

print(len(data[mask_credit_score]))

# g00d = 2550
# fair = 2544
# low = 2534

2534


In [52]:
print(len(data.columns))

15


In [53]:
churned = []
unchurned = []

for i in range(0, len(y)):
  if y[i] == 0:
    unchurned.append(y[i])
  else:
    churned.append(y[i])

In [54]:
print(len(churned), len(unchurned))

9976 24


In [55]:
credit_score = data.iloc[:, 3].values

In [56]:
print(credit_score.max())

850


In [57]:
from sklearn.preprocessing import OrdinalEncoder
ord_encoder = OrdinalEncoder()
X[:, 2:3] = ord_encoder.fit_transform(X[:, 2:3])

In [58]:
print(X)

[[619 'France' 0.0 ... 1 101348.88 1]
 [608 'Spain' 0.0 ... 1 112542.58 0]
 [502 'France' 0.0 ... 0 113931.57 1]
 ...
 [709 'France' 0.0 ... 1 42085.58 1]
 [772 'Germany' 1.0 ... 0 92888.52 1]
 [792 'France' 0.0 ... 0 38190.78 0]]


In [59]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [60]:
print(X)

[[1.0 0.0 0.0 ... 1 101348.88 1]
 [0.0 0.0 1.0 ... 1 112542.58 0]
 [1.0 0.0 0.0 ... 0 113931.57 1]
 ...
 [1.0 0.0 0.0 ... 1 42085.58 1]
 [0.0 1.0 0.0 ... 0 92888.52 1]
 [1.0 0.0 0.0 ... 0 38190.78 0]]


In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [62]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Generation
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1,3
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,3
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,3
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0,3
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,2


In [63]:
X_train[:, :]

array([[0.0, 1.0, 0.0, ..., 1, 124749.08, 0],
       [1.0, 0.0, 0.0, ..., 0, 41104.82, 0],
       [0.0, 1.0, 0.0, ..., 1, 45750.21, 1],
       ...,
       [1.0, 0.0, 0.0, ..., 1, 92027.69, 1],
       [1.0, 0.0, 0.0, ..., 1, 101168.9, 0],
       [0.0, 1.0, 0.0, ..., 0, 33462.94, 1]], dtype=object)

In [64]:
X_train.shape[1]

13

In [65]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [66]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(X_train, y_train)

RandomForestClassifier()

In [67]:
y_pred = classifier.predict(X_test)

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

In [69]:
inputs = keras.Input(shape=(X_train.shape[1],), name="Churn Features")
layer_1 = layers.Dense(32, activation="relu", name="dense_layer_1")(inputs)
layer_2 = layers.Dense(32, activation="relu", name="dense_layer_2")(layer_1)
layer_3 = layers.Dense(32, activation="relu", name="dense_layer_3")(layer_2)
outputs = layers.Dense(2, activation="sigmoid", name="output_layer")(layer_3)

In [70]:
model = keras.Model(inputs=inputs, outputs=outputs, name="Bank_Churn_Model_1")

In [71]:
model.compile(optimizer=keras.optimizers.RMSprop(),
              loss= keras.losses.SparseCategoricalCrossentropy(),
              metrics=[keras.metrics.mae, keras.metrics.sparse_categorical_accuracy])

In [72]:
model.summary()

Model: "Bank_Churn_Model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Churn Features (InputLayer  [(None, 13)]              0         
 )                                                               
                                                                 
 dense_layer_1 (Dense)       (None, 32)                448       
                                                                 
 dense_layer_2 (Dense)       (None, 32)                1056      
                                                                 
 dense_layer_3 (Dense)       (None, 32)                1056      
                                                                 
 output_layer (Dense)        (None, 2)                 66        
                                                                 
Total params: 2626 (10.26 KB)
Trainable params: 2626 (10.26 KB)
Non-trainable params: 0 (0.00 Byte)
______________

In [73]:
"""
print("Fitting the model on the training data")
history = model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=100
)

"""

'\nprint("Fitting the model on the training data")\nhistory = model.fit(\n    X_train,\n    y_train,\n    batch_size=32,\n    epochs=100\n)\n\n'

In [74]:
#y_pred = model.predict(X_test)
#y_pred = (y_pred > 0.5)

In [75]:
"""
test_scores = model.evaluate(X_test, y_test, verbose=2)
print("Test Loss:", test_scores[0])
print("Test accuracy:", test_scores[1])
dict(zip(model.metrics_names, test_scores))
"""

'\ntest_scores = model.evaluate(X_test, y_test, verbose=2)\nprint("Test Loss:", test_scores[0])\nprint("Test accuracy:", test_scores[1])\ndict(zip(model.metrics_names, test_scores))\n'

In [76]:
print(cm)

[[   0    3    0    0    0]
 [   0  113    0    0    0]
 [   0    0  489    0    0]
 [   0    0    0 1233    0]
 [   0    0    0    0  162]]


In [77]:
print(acc)

0.9985


In [78]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = classifier.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

ValueError: ignored

In [ ]:
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
#plt.xlim([0, 1])
#plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()